In [3]:
import numpy as np
import pandas as pd
import torch 
import tensorboard

Generate dataset

- Consider the exponential function f(x)=exp(-x/c), with x=0,1,2,….,L-1 integers
- Generate many training datasets {f(0,f(1),f(2),…f(L-1)} for different values of c
- Now we solve the inverse problem: input c and see if you get the exponential.

In [69]:
#Returns a dataframe of calculated exponential functions for given C-parameters and L-angular quantum numbers
class GenerateData():
    def __init__(self, l):
        self.l = l
        self.x = np.arange(l)
        self.c = np.arange(start=0.0001, stop=1, step = 0.0001)
    
    #Evaluate function for given c-values
    def Func(self, input, arg):
        #func= np.zeros(shape = (self.l,))
        #for i in range(0, self.l):
        #    func[i] = np.e**(-self.x[i]/arg)
        return np.e**(-input/arg)
    
    #Create dataset: 100,000 funcs?
    def Populate(self):
        func_df = pd.DataFrame(index = self.c.tolist())
        for i in self.x:
            col = []
            for j in self.c:
                col.append(GenerateData(self.l).Func(i,j))
            func_df[i] = col
        func_df['c'] = self.c
        return func_df
    
    #Set param value for integer L-value (angular momentum quantum number)
    def Data(self):
        return GenerateData(self.l).Populate()

Dataframe for Exponential Function Values: Rows = C-values, Columns = Integer X-values

In [72]:
data = GenerateData(4).Data()
data

,0,1,2,3,c
0.0001,1.0,0.000000,0.000000,0.000000,0.0001
0.0002,1.0,0.000000,0.000000,0.000000,0.0002
0.0003,1.0,0.000000,0.000000,0.000000,0.0003
0.0004,1.0,0.000000,0.000000,0.000000,0.0004
0.0005,1.0,0.000000,0.000000,0.000000,0.0005
...,...,...,...,...,...
0.9995,1.0,0.367695,0.135200,0.049712,0.9995
0.9996,1.0,0.367732,0.135227,0.049727,0.9996
0.9997,1.0,0.367769,0.135254,0.049742,0.9997
0.9998,1.0,0.367806,0.135281,0.049757,0.9998


Initialize neural net

In [68]:
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor, Lambda
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


Linear stack NN with a Softmax screen - initiate with nodes equal to L-value

In [96]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(len(data.columns), 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(30, 3),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=5, out_features=100, bias=True)
    (2): ReLU()
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): ReLU()
    (5): Linear(in_features=30, out_features=3, bias=True)
  )
)


Implementation of Dataloader for Tensor Values

In [93]:
#Drop indicies for labels
drop_vals = np.arange(len(data.columns)-1).tolist()
for i in drop_vals:
    drop_vals[i] = str(drop_vals[i])

training_data = ToTensor(data.drop(columns='c'))
test_data = ToTensor(data.drop(drop_vals))

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

TypeError: ToTensor.__init__() takes 1 positional argument but 2 were given

Model Params

In [94]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)



Model Training/Testing

In [95]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

Sample Testing of Network on Random Tensors

In [19]:
X = torch.rand(100, 1, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1])
